In [ ]:
# Installations
!pip install emot
!pip install nltk
!pip install PyDictionary
print("Installations complete")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 20 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for goslate: filename=goslate-1.5.4-py3-none-any.whl size=11597 sha256=161583c0146d0a81aa71fc86691859194fdf275e72e98ae35825ff293fdf5d6e
  Stored in directory: /root/.cache/pip/wheels/7e/08/d5/88fc81ecccbff86db9b3dac7ca657c2cc64b5d10db421bd2a2
Successfully built goslate


Installations complete


In [ ]:
# Libraries to import
from google.colab import drive
import numpy as np
import pandas as pd
import re
import emot
import os
import nltk
nltk.download('words')
nltk.download('punkt')
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from PyDictionary import PyDictionary
from nltk.stem import PorterStemmer
print("Libraries have been successfully imported")

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Libraries have been successfully imported


In [ ]:
# Ensure you make a shortcut of BT4013 CAPSTONE on own drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Paths
training_set_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/sensitised_IG_RnR_training_dataset.xlsx"
emoji_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/emoji.xlsx"
emoticon_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/emoticons.xlsx"
nonenglish_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/nonenglish.xlsx"

test_set_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/sensitised_IG_RnR_datasets_SingBert_labelled.xlsx"
test_emoji_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/test_emoji.xlsx"
test_emoticon_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/test_emoticons.xlsx"
test_nonenglish_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/test_nonenglish.xlsx"

In [ ]:
# Import Training dataset
data = pd.read_excel(training_set_path, index_col=0)
data.head()

,page,text,label
0.0,0.0,"@bgAABHtNhL is synonymous with Girlfriends, a ...",others
1.0,0.0,"A Malay Muslim desktop engineer by trade, @bgA...",others
2.0,0.0,"A Malay Muslim desktop engineer by trade, @bgA...",others
3.0,0.0,We're celebrating this July! 🎉 \n \nJoin our C...,others
4.0,0.0,"Be the change you want to see, they say. How d...",others


In [ ]:
emoji_engine = emot.core.emot()
emoji_engine.bulk_emoji(["@bgAABHAPTM.tm_ 😂😂"])

[{'value': ['😂', '😂'],
  'location': [[16, 17], [17, 18]],
  'mean': [':face_with_tears_of_joy:', ':face_with_tears_of_joy:'],
  'flag': True}]

In [ ]:
# ALL FUNCTIONS WILL RESIDE IN THIS CODE TAB
def icons_to_words(isEmoji:bool, dataset_path:str, text_col:str) -> dict:
  '''
  This function finds all emojis/emoticons in a chunk of text and assigns a default meaning to it
  Returns: A dictionary of an emoji/emoticon with its assigned meaning
  '''
  data = pd.read_excel(dataset_path, index_col=0)
  emoji_engine = emot.core.emot()
  storage = {}
  text_list = data[text_col].tolist()
  breakdown = list(map(lambda x: (x['value'], x['mean']), emoji_engine.bulk_emoji(text_list))) if isEmoji else list(map(lambda x: (x['value'], x['mean']), emoji_engine.bulk_emoticons(text_list)))

  for emoji_list, mean_list in breakdown:
    for i in range(len(emoji_list)):
      if emoji_list[i] not in storage:
        storage[emoji_list[i]] = mean_list[i].replace(':', "").replace('_', " ")
  return storage


def update_external_corpus(path:str, column_names:list, corpus:dict) -> None:
  '''
  Updates the external xlsx corpus files with newly added words
  '''
  corpus_present = os.path.isfile(path)
  if not corpus_present:
    temp_df = pd.DataFrame(columns=column_names)
    for feature, desc in corpus.items():
      temp_df = temp_df.append({column_names[0]:feature, column_names[1]:desc}, ignore_index=True)
  else:
    temp_df = pd.read_excel(path)
    features = set()
    for feature in temp_df[column_names[0]]:
      features.add(feature)
    for feature, desc in corpus.items():
      if feature not in features:
        temp_df = temp_df.append({column_names[0]:feature, column_names[1]:desc}, ignore_index=True)
  temp_df.to_excel(path, index=False)


def convert_emojis_to_word(text:str, storage:dict) -> str:
  '''
  Function used to preprocess emojis to words
  '''
  for emot in storage:
    if text is not np.nan:
      text = re.sub(r'('+emot+')', " ".join(storage[emot].split()), text)
  return text


def convert_emoticons_to_words(text:str, storage:dict) -> str:
  '''
  Function used to preprocess emoticons to words
  '''
  for emot in storage:
    if text is not np.nan:
      text = text.replace(emot, storage[emot])
  return text


def find_sus_english_words(storage:dict, text:str, ps=PorterStemmer()):
  '''
  Use NLTK to shortlist possible words that are not in English
  '''
  text_list = word_tokenize(text)
  for token in text_list:
    if ps.stem(token) not in setofwords and token not in storage and token.isalpha():
      storage[token] = token



def find_non_english_words(storage:dict, book=PyDictionary()):
  '''
  Using an online scraper to check and confirm if word is not English, VERY EXPENSIVE
  '''
  temp_store = {}
  for token,_ in storage.items():
    if not bool(book.meaning(token)):
      temp_store[token] = token
  return temp_store


def preprocessing_engine(dataset_path:str, text_col:str):
  '''
  PREPROCESSING STEPS
  1. Replace emojis with words
  2. Remove all <@usernames>
  3. Remove all <#Hashtags>
  4. Remove all URLS (Links beginning with http or https)
  5. Remove all emails
  6. Replace emoticons with words.
  7. Change all letters to lower case
  8. Remove all new lines, new tabs or carriage returns
  9. Ensure there is only 1 whitespace between words
  '''
  data = pd.read_excel(dataset_path, index_col=0) # Temporary fix to training set issues
  data[text_col] = data[text_col].apply(lambda x: convert_emojis_to_word(x, emoji_dict))
  data[text_col] = data[text_col].str.replace("\B\@\S+ |\B\@\S+", "", regex=True)
  data[text_col] = data[text_col].str.replace("\B\#\S+ |\B\#\S+", "", regex=True)
  data[text_col] = data[text_col].str.replace('(?:https?|ftp):\/\/[\w/\-?=%.]+\.[\w/\-&?=%.]+', "", regex=True)
  data[text_col] = data[text_col].str.replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', "", regex=True)
  data[text_col] = data[text_col].apply(lambda x: convert_emoticons_to_words(x, emoticons_dict))
  data[text_col] = data[text_col].str.lower()
  data[text_col] = data[text_col].str.replace('\r+|\n+|\t+',' ', regex=True)
  data[text_col] = data[text_col].apply(lambda x: " ".join(x.split()) if x is not np.nan else x)
  return data

In [ ]:
###### DIAGNOSTIC: PLEASE SET THE CORRECT PATHS BEFORE RUNNING THE FOLLOWING FUNCTIONS ######
target_dataset_path = test_set_path
target_emoji_path = emoji_dict_path
target_emoticons_path = emoticon_dict_path
target_shortform_path = test_nonenglish_dict_path

In [ ]:
# Core logic of emoji / emoticions -> words
emoji_dict = icons_to_words(True, target_dataset_path, 'text')
update_external_corpus(target_emoji_path, ["Emoji", "Meaning"], emoji_dict)
print(emoji_dict)

{'👋': 'waving hand', '🎉': 'party popper', '👈': 'backhand index pointing left', '✨': 'sparkles', '🎨': 'artist palette', '🙏': 'folded hands', '🙌': 'raising hands', '🌙': 'crescent moon', '✅': 'check mark button', '✏': 'pencil', '📸': 'camera with flash', '✊🏽': 'raised fist medium skin tone', '❤': 'red heart', '🌸': 'cherry blossom', '🌈': 'rainbow', '🎙': 'studio microphone', '😔': 'pensive face', '🙄': 'face with rolling eyes', '😢': 'crying face', '💬': 'speech balloon', '📖': 'open book', '🤦🏻\u200d♀️': 'woman facepalming light skin tone'}


In [ ]:
emoticons_dict = icons_to_words(False, target_dataset_path, 'text')
update_external_corpus(target_emoticons_path, ["Emoticon", "Meaning"], emoticons_dict)
print(emoticons_dict)

{':/': 'Skeptical, annoyed, undecided, uneasy or hesitant', 'd:': 'Tongue sticking out, cheeky, playful or blowing a raspberry', ';)': 'Wink or smirk', ':)': 'Happy face or smiley', '=D': 'Laughing, big grin or laugh with glasses', ':3': 'Happy face smiley', ';D': 'Wink or smirk', 'QQ': 'Sad or Crying', 'DX': 'Great dismay', 'oO': 'Surprised', '=3': 'Laughing, big grin or laugh with glasses', '=p': 'Tongue sticking out, cheeky, playful or blowing a raspberry', 'XP': 'Tongue sticking out, cheeky, playful or blowing a raspberry', ':>': 'Happy face smiley', '0:3': 'Angel, saint or innocent', 'XD': 'Laughing, big grin or laugh with glasses', ':(': 'Frown, sad, andry or pouting', ':-(': 'Frown, sad, andry or pouting', ':o': 'Surprise', '=/': 'Skeptical, annoyed, undecided, uneasy or hesitant', '8D': 'Laughing, big grin or laugh with glasses', '>:)': 'Evil or devilish', '=L': 'Skeptical, annoyed, undecided, uneasy or hesitant', ':<': 'Frown, sad, andry or pouting', ":')": 'Tears of happiness

In [ ]:
processed_dataset = preprocessing_engine(target_dataset_path, 'text')
processed_dataset.head()

,group,id,text,time,likes,comments,type,post_id,comment_id,text_pred,text_pred_prob,text_label
0.0,0.0,b'gAAAAABjHsN3lwZhNGi-AiGMiKAlaqsf5Okowm6bY8XY...,"in part 2 of her article, n.h rahman analyses ...",2021-12-18 14:58:28,35.0,0.0,Post,NaN,NaN,2.0,0.999916,social activism
1.0,1.0,b'gAAAAABjHsN3jpTi8BHHJ22-VFogHsPfs0xtvStni-tm...,hidayah amin is a force to be reckoned with. h...,2021-12-15 18:00:33,100.0,0.0,Post,NaN,NaN,3.0,0.999840,solidarity
2.0,1.0,b'gAAAAABjHsN39aHFT9kD5vWMDdDeyoSxhOBV207WC_Pr...,"this year, we get up close and personal with t...",2021-12-14 18:01:04,72.0,2.0,Post,NaN,NaN,1.0,0.991582,others
3.0,2.0,b'gAAAAABjHsN3Vlhtr6w4HR1DrSzzLaUZREcBhwJHYt3j...,edwin tong strikes again,2021-12-14 09:59:12,2516.0,9.0,Post,NaN,NaN,3.0,0.999707,solidarity
4.0,2.0,b'gAAAAABjHsN3tRg6EgulC0KBsGRn02OFptSKHsQjGqlc...,congratulations loh kean yew! he just best the...,2021-12-14 00:48:54,3519.0,31.0,Post,NaN,NaN,1.0,0.983010,others


In [ ]:
# If shortforms corpus already exists, then this step is not necessary to run as it is very expensive

setofwords = set(words.words())
non_english = {}

processed_dataset['text'].apply(lambda x: find_sus_english_words(non_english, x) if x is not np.nan else x)

update_external_corpus(target_shortform_path, ["Word", "Meaning"], find_non_english_words(non_english))